In [ ]:
import os
HOME = os.getcwd()
#print(HOME)
%cd {HOME}
!git clone https://github.com/IDEA-Research/GroundingDINO.git
%cd {HOME}/GroundingDINO
%pip install datumaro
%pip install -r requirements.txt
%pip install -q -e .


In [ ]:
import os

CONFIG_PATH = os.path.join(HOME, "GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py")
print(CONFIG_PATH, "; exist:", os.path.isfile(CONFIG_PATH))

In [ ]:
%pip install tensorrt --extra-index-url https://pypi.nvidia.com

In [ ]:
%cd {HOME}
print(HOME)
%cd {HOME}/weights

!wget -q https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha/groundingdino_swint_ogc.pth

In [ ]:
import os

WEIGHTS_NAME = "groundingdino_swint_ogc.pth"
WEIGHTS_PATH = os.path.join(HOME, "weights", WEIGHTS_NAME)
print(WEIGHTS_PATH, "; exist:", os.path.isfile(WEIGHTS_PATH))

In [ ]:
%cd /home/jupyter/datasphere/project/GroundingDINO

from groundingdino.util.inference import Model, load_model, load_image, annotate

model = Model(model_config_path=CONFIG_PATH, model_checkpoint_path=WEIGHTS_PATH)

In [ ]:
import os
import supervision as sv
import torch
import cv2
import numpy as np
import PIL
from tqdm import tqdm
from datumaro import Polygon, Bbox, RleMask, Dataset, DatasetItem, Image
import warnings

warnings.filterwarnings("ignore")

images = []
frame_path = "/home/jupyter/datasphere/project/datata/Frames"

for _,_,files in os.walk(frame_path):
    images = files

for i,im in enumerate(images):
    images[i] = frame_path + "/" + im 

    

    

results = []
for image_name in tqdm(images):
    image_path = os.path.join(HOME, "data", image_name)

    CLASSES = ["car","bus","truck"]
    BOX_TRESHOLD = 0.35
    TEXT_TRESHOLD = 0.25


    image = cv2.imread(image_name)

    detections = model.predict_with_classes(
        image=image,
        classes=CLASSES,
        box_threshold=BOX_TRESHOLD,
        text_threshold=TEXT_TRESHOLD
    )
    boxes = detections.xyxy
    
    
    
    box_annotator = sv.BoxAnnotator()
    labels = [
        f"{CLASSES[class_id]} {confidence:0.2f}" 
        for _, _, confidence, class_id,_ in detections]
    annotated_frame = box_annotator.annotate(scene=image.copy(), detections=detections, labels=labels)

    #matplotlib inline
    #sv.plot_image(annotated_frame, (16, 16))


    h, w, _ = image.shape

    annotations = []
    ids  = [class_id for _, _, confidence, class_id,_ in detections]
    for i,box in enumerate(boxes):
        result = torch.from_numpy(box) * torch.Tensor([w, h, w, h])
        annotations.append(Bbox(result[0], result[1], result[2], result[3], label=ids[i]))
    
    results.append(
        DatasetItem(
            id=image_name.split(".")[0],
            media=Image.from_file(path=image_path),
            annotations=annotations
            )
        )
    
    try:
        name = image_name.split("/")[-1]
        
        PIL.Image.fromarray(annotated_frame).save('/home/jupyter/datasphere/project/GroundingDINO/coco_export.json/images' + "/" + name)
    except Exception as e:
        print(e)
        pass
    
    
dataset = Dataset.from_iterable(results,categories = CLASSES)


In [ ]:
dataset.export("./coco_export.json",format="coco")

In [ ]:
import json
import os
from tqdm import tqdm
import numpy as np

coco_file = "/home/jupyter/datasphere/project/GroundingDINO/coco_export.json/annotations/instances_default.json" 
def convert_coco_to_yolov8(coco_file):
    with open(coco_file) as f:
        coco = json.load(f)

        images = coco['images']
        annotations = coco['annotations'] 
        categories = {cat['id']: cat['name'] for cat in coco['categories']}
        #print(categories)
        

        os.makedirs('yolov8_labels', exist_ok=True)

        for image in tqdm(images, desc='Converting images'):
            image_id = image['id']
            filename = image['file_name']
            label_filename = (filename.split('.jpg')[0]).split('/')[-1]
            
            label_path = os.path.join('yolov8_labels', f'{label_filename}.txt')

            with open(label_path, 'w') as f:
                for ann in annotations:
                    
                    if ann['image_id'] != image_id:
                        continue
                    #print(ann)
                    img_width = image['width']
                    img_height = image['height']
                    #print(img_width)
                    #print(f"Widht={img_width},height={img_height}")
                    label_id = ann['category_id']
                    #print(label_id)
                    label = categories[label_id]

                    x1, y1, w, h = ann['bbox']
                    
                    x1 = x1/img_width
                    y1 = y1/img_height
                    w = w/img_width
                    h = h/img_height
                    #print(x1, '  ', y1,'  ', w, "  ", h)
                    
                    x_center=((2*x1+w)/(2*img_width))
                    y_center=((2*y1+h)/(2*img_height))
                    w_box=w/img_width
                    h_box=h/img_height
                    
                    if (x_center>=1) or (y_center>=1):
                        break
                    
                    segmentation_points_str = '{:f} {:f} {:f} {:f}'.format(x_center,y_center,w_box,h_box)
                    #print(segmentation_points_str)

                    line = '{} {}'.format(label_id, segmentation_points_str)
                    #print(line)
                    
                    f.write(line + '\n')
                
convert_coco_to_yolov8(coco_file)